# PACKAGE IMPORT


In [ ]:
import get_data as gd
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import Asset as a
import numpy as np

In [ ]:
#grab_data
df_data = gd.grab_data()

#Hyperparameter
#Adjusted accordingly to compare with proposed strategy
start_time = 360
end_time = 450

df_data = df_data[start_time:end_time]

# Baseline Functions

In [ ]:
days = range(start_time, end_time)

def baselineFunction(ema, sma):
    assetBase = a.Asset()


    in_position = False
    ema = np.asarray(df_data[ema])
    sma = np.asarray(df_data[sma])
    ema_vs_sma = ema > sma #gets indexes when ema is larger tan sma
    money_progress = [] # list to store money every day

    btc = np.asarray(df_data['close'])

    in_position = False
    for i in range(len(df_data)):
        #buy trigger
        if not in_position and ema_vs_sma[i]:
            assetBase.buy(btc[i])
            in_position = True
        # Sell trigger
        elif in_position and not ema_vs_sma[i]:
            assetBase.sell(btc[i])
            in_position = False

        money_progress.append(assetBase.coin * btc[i] + assetBase.money)

    #Final sell at the end of period
    if in_position:
        assetBase.sell(btc[-1])

    return money_progress

In [ ]:
import matplotlib.pyplot as plt

#market condition
place_per_bitcoin = np.asarray(df_data['close'])

fig, ax1 = plt.subplots()
fig.subplots_adjust(right=0.85 )

#trade_strategies 
ema21_sma21 = baselineFunction('ema_21', 'sma_21')
ema9_sma9 = baselineFunction('ema_9', 'sma_9')
ema21_sma9 = baselineFunction('ema_21', 'sma_9')
ema9_sma21 = baselineFunction('ema_9', 'sma_21')

# Define plots
ax1.plot(days,ema9_sma9, color='r', label='ema9_sma9')
ax1.plot(days,ema21_sma21, color='b', label='ema21_sma21')
ax1.plot(days,ema21_sma9, color='k', label='ema21_sma9')
ax1.plot(days,ema9_sma21, color='g', label='ema9_sma21')

#bitcoin market plot
ax2 = ax1.twinx()
ax2.plot(days, price_per_bitcoin, linestyle='--', color='orange', alpha=0.5, label='BTC pice (AUD')

#Title and axis
ax1.legend()
plt.title('Bitcoin trading using EMA vx SMA strategy')
ax1.set_xlabel('Day')
ax1.set_ylabel('Money(AUD')
ax2.set_ylabel('BTC Price (AUD)')
plt.savefig(f'figure/{start_time}to{end_time}BaselineStrategy.png')
plt.show()
